In [ ]:
contour_positions_file = r""
output_file = r""

fov_length = 200 # The length of the field of view in micro-meter

In [ ]:
import pandas as pd
import numpy as np
from shapely import geometry
from shapely.ops import cascaded_union

import matplotlib.pyplot as plt

In [ ]:
# Load the contour positoins
contour_positions = np.array(pd.read_csv(contour_positions_file, header = None, sep=','))

In [ ]:
# Initialize the grid
x_min = np.min(contour_positions[:,0])
y_min = np.min(contour_positions[:,1])
x_max = np.max(contour_positions[:,0])
y_max = np.max(contour_positions[:,1])

grid = []
x_positions = np.arange(x_min - fov_length, x_max + fov_length, fov_length)
y_positions = np.arange(y_min - fov_length, y_max + fov_length, fov_length)

for i in range(len(x_positions)):
    for j in range(len(y_positions)):
        if i % 2 == 0:
            grid.append([x_positions[i], y_positions[j]])
        else:
            grid.append([x_positions[i], y_positions[len(y_positions) - j - 1]])
            
grid = np.array(grid)

In [ ]:
# Select the grids within the contour
selected_grid = []
area_of_interest = geometry.Polygon(contour_positions)

r = fov_length / 2
for i in range(len(grid)):
    tile_corners = [[grid[i][0] - r, grid[i][1] - r], [grid[i][0] - r, grid[i][1] + r],
                    [grid[i][0] + r, grid[i][1] + r], [grid[i][0] + r, grid[i][1] - r]]
    tile = geometry.Polygon(tile_corners)
    
    if area_of_interest.intersects(tile):
        selected_grid.append(grid[i])
        
selected_grid = np.array(selected_grid)

In [ ]:
# Plot the selected position grid
plt.scatter(selected_grid[:,0], selected_grid[:,1])
plt.scatter(contour_positions[:,0], contour_positions[:,1])
plt.show()

In [ ]:
# Save the selected position grid
np.savetxt(output_file, selected_grid, delimiter=',')